In [7]:
# This one seems to be fully automated by me
# Just copy-paste it when the time comes
# copy the file:
import shutil  # for file manipulation
import os
import win32com.client as win32
from ok_sap_script import *
from datetime import datetime
import time

class Control():
    def __init__(self):
        self.curr_user = os.getlogin()
        self.final_path_to_prev = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Planned_price/Planned_price_model.xlsx" # model file
        self.new_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Planned_price/Planned_price_for_{datetime.today().strftime('%d-%m-%Y')}.xlsx"
        
        # SAP part
        self.sap_path_for_extract = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Planned_price/Output"
        self.sap_file_name = f"Extract_sap_for_{datetime.today().strftime('%d-%m-%Y')}.XLSX"
        self.sap_path_full = f"{self.sap_path_for_extract}/{self.sap_file_name}"

        self.period = "07"
        self.formula_comment = ""

        # SAP variables
        self.environment = "ECC Production"
        self.t_code_sap = "ZP2M_TRACKSTDP"
        self.variant_sap = "PP_AUTO"

        # emails
        self.email_receiver = "oleksandr.komarov@zoetis.com"

    def sap_extract(self):
        # ensure there are no open sessions
        sap_close()
        # open new session
        open_sap = sap_open()
        time.sleep(5)
        if open_sap == True:
            session = sap_logon(environment=self.environment, client=1)
            sap_code(tcode=self.t_code_sap, session=session)
            try:
                sap_variant(session=session, var_to_use=self.variant_sap)
            except: 
                sap_variant(session=session, var_to_use=self.variant_sap, version=2) # in our case, V2 is the most probable scenario
            # run the report
            sap_run(session=session)
            sap_extract(session=session, extr_path=f"C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Planned_price\\Output\\", extr_name=self.sap_file_name)
            sap_close()
            close_excel()
            

    def copy_file(self):
        try:
            shutil.copy(self.final_path_to_prev, self.new_file_path)
        except PermissionError as e:
            print("The file may be opened")
            os.system("taskkill /f /im  excel.exe")
    # open created Excel file and perform the following manipulations
        try:
            # default Excel runtime optimisation
            excel = win32.Dispatch("Excel.Application")
            excel.AskToUpdateLinks = False
            excel.DisplayAlerts = False
            excel.Visible = True
            excel.ScreenUpdating = False
            # open new file and disable calculations
            self.new_file = excel.Workbooks.Open(self.new_file_path)
            excel.Application.Calculation = (-4135)  # to set xlCalculationManual # Workbook needs to be opened
    # other steps
                # manipulate sap extract file
                    # open sap extract file
            self.sap_file = excel.Workbooks.Open(self.sap_path_full)
                    # find last row of this file
            self.sap_file_ws = self.sap_file.Worksheets(1)
            self.last_row_sap_file_ws = self.sap_file_ws.Cells(1, 12).End(-4121).Row # column 12 -> L is because there is a sum there
                # manipulate Extract sheet in the model file
            self.new_file_extract_ws = self.new_file.Sheets(f"Extract")
                    # clear contents of that sheet
            self.new_file_extract_ws.Range("A:Y").ClearContents()
                    # paste from sap file to this file
            self.sap_file_ws.Range(f"A1:X{self.last_row_sap_file_ws-1}").SpecialCells(12).Copy(Destination=self.new_file_extract_ws.Range(f"A1"))
                    # add comments column
            self.new_file_extract_ws.Range("Y1").Value = "Comments"
                    # apply formula for the most frequent causes
            self.new_file_extract_ws.Range(f"Y2:Y{self.last_row_sap_file_ws-1}").Formula = '=IF(H2="DI", "Discontinued>no risk", IF(H2="DF", "DF > no risk as it is in initialization state", IF(LEFT(A2,1)="4","Raw material, no transactions > no risk", IF(LEFT(A2,1)="5","Reached out to MMDMO as no costing tabs in MM03, MMDMO confirmed that other 5 code SKUs have no pricing", ""))))'
                        # remaining cases are for the user to analyse manually -> maybe check prev file at a later stage
                    
            # close all other files
            self.sap_file.Close()
            self.new_file.Save()           
            # back to default
            excel.ScreenUpdating = True
            excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
            # close file
            self.new_file.Close()

            # send this final file to the end user by email
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = self.email_receiver
            mail.Subject = f"Automatic email for Planned price at 0"
            mail.Body = f"Please find attached a file with SAP extract as of {datetime.today().strftime('%d-%m-%Y')}"
            mail.Attachments.Add(self.new_file_path)
            mail.Send()

        except Exception as e:
            excel.ScreenUpdating = True
            excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
            print(f"something went wrong, namely\n{e}")

control = Control()
control.sap_extract()
control.copy_file()



In [2]:
# This function is for testing material price determination setting in SAP
from ok_sap_script import *
import win32com.client as win32
import time
import os
from datetime import datetime

def sap_part():
    # outlook
    receiver = "oleksandr.komarov@zoetis.com"
    curr_user = os.getlogin()
    extract_path = f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Price_determination/Output"
    extract_name = f"PD-{datetime.today().strftime('%d-%m-%Y')}.xlsx"

    environment = "ECC (QE3)"
    t_code_sap = "se16"
    table_sap = "mbew"
    variant_sap = "MBEW_PD"
    variant_sap2 = "/MBEW_NB"
    # ensure there are no open sessions
    close_sap = sap_close()
    # open new session
    open_sap = sap_open()
    time.sleep(5)
    if open_sap == True:
        session = sap_logon(environment=environment, client=1)
        sap_code(tcode=t_code_sap, session=session)
        # enter mbew
        session.findById("wnd[0]/usr/ctxtDATABROWSE-TABLENAME").text = table_sap
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]/usr/ctxtI1-LOW").text = "10000000"
        session.findById("wnd[0]/usr/ctxtI1-HIGH").text = "19999999"
        session.findById("wnd[0]/usr/txtMAX_SEL").text = "10000000"
        session.findById("wnd[0]").sendVKey(8)

        # select layout
        sap_layout(session=session, layout=variant_sap)
        # check if there is no output
        mbew_body = session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")
        rows = mbew_body.rowCount
        if rows !=0:
            print("There is some data. Export")
            # extract
            sap_extract(session=session, extr_path=extract_path, extr_name=extract_name)
            close_excel()
            # send by email
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = receiver
            mail.Subject = f"Automatic email for price determination control"
            mail.Body = f"Please consult the attachment"
            mail.Attachments.Add(f"{extract_path}/{extract_name}")
            #mail.Display(True)
            mail.Send()
            sap_close()
        else:
            print("There is no data. End here")
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = receiver
            mail.Subject = f"Automatic email for price determination control"
            mail.Body = f"There are no materials to be corrected"
            #mail.Display(True)
            mail.Send()
            sap_close()
       


sap_part()

There is some data. Export


In [52]:
# This function is for testing TP montly emails in SAP
# Test SAP part

from ok_sap_script import *
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os
import win32com.client as win32

curr_user = os.getlogin()
pi01_path = f"pi01-{datetime.today().strftime('%d-%m-%Y')}.xlsx"
ziv1_path = f"ziv1-{datetime.today().strftime('%d-%m-%Y')}.xlsx"

def sap_extract_me():   
    first_day = datetime.today().replace(day=1).strftime("%d.%m.%Y")
    environment = "ECC (QE3)"
    t_code_sap = "ZP2M_OTBD_SP_FL_NONF"
    variant_flash = "FLASH_ME"
    variant_non_flash = "NON_FLASH_ME" 
    

    # ensure there are no open sessions
    close_sap = sap_close()
    # open new session
    open_sap = sap_open()
    time.sleep(5)
    if open_sap == True:
        session = sap_logon(environment=environment, client=1)
        sap_code(tcode=t_code_sap, session=session)
        try:
            sap_variant(session=session, var_to_use=variant_flash)
        except: 
            sap_variant(session=session, var_to_use=variant_flash, version=2) # in our case, V2 is the most probable scenario

        session.findById("wnd[0]/usr/ctxtS_DATBI-LOW").text = first_day
        # run the report
        sap_run(session=session)
        sap_extract(session=session, extr_path="C:\\Users\\KOMAROVO\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Files_extracted\\", extr_name=pi01_path)
        close_sap = sap_close()
    # Do the same but for non-flash
        # open new session
    open_sap = sap_open()
    time.sleep(5)
    if open_sap == True:
        session = sap_logon(environment=environment, client=1)
        sap_code(tcode=t_code_sap, session=session)
        try:
            sap_variant(session=session, var_to_use=variant_non_flash, version=1)
        except: 
            sap_variant(session=session, var_to_use=variant_non_flash, version=2) # in our case, V2 is the most probable scenario

        session.findById("wnd[0]/usr/ctxtS_DATB1-LOW").text = first_day # this one differs
        # Run the report
        sap_run(session=session)
        sap_extract(session=session, extr_path="C:\\Users\\KOMAROVO\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Files_extracted", extr_name=ziv1_path)
        close_sap = sap_close()
        close_excel()

def excel_rework_me():
    # flash
    #close_excel()
    global list_LEs
    list_LEs = []
    # grab with pandas, loop through, populate separete files in Files_spit, attach to email later
    df_pi01 = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Output/Files_extracted/{pi01_path}")
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="flash_copy")
    for index, row in df_email_contacts.iterrows():
        df_pi01_split = df_pi01[df_pi01['Sales Organization'] == row['LE']]
        if df_pi01_split.empty: # if no record for particular market exist
            pass
        else:
            save_to = f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Output/Files_split/{row['LE']}-{pi01_path}"
            df_pi01_split.to_excel(save_to, index=False)
            list_LEs.append(row['LE'])
            send_emails_me_flash(save_to, row['LE'])
    # non flash
    # do the same for ziv1 file
    df_ziv1 = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Output/Files_extracted/{ziv1_path}")
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="non_flash_copy")
    for index, row in df_email_contacts.iterrows():
        df_pi01_split = df_ziv1[df_ziv1['Customer Number'] == row['Customer LE']]
        if df_pi01_split.empty: # if no record for particular market exist
            pass
        else:
            save_to = f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Output/Files_split/{row['LE']}-{ziv1_path}"
            df_pi01_split.to_excel(save_to, index=False)
            list_LEs.append(row['LE'])
            send_emails_me_non_flash(save_to, row['LE'])
    save_last_x_emails()
    
def save_last_x_emails():
    number_emails = len(list_LEs)
    # save files
    outlook = win32.Dispatch('outlook.application')
    mapi = outlook.GetNameSpace("MAPI")
    # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
    folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
    items= folder.Items
    items.Sort("[ReceivedTime]", Descending=True)
    msgs = items.GetFirst()
    msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\{msgs.Subject}.msg")
    for _ in range(number_emails - 1):
        msgs = items.GetNext()
        msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\{msgs.Subject}.msg")
    print(msgs)


def send_emails_me_flash(save_path, cocd):
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="flash_copy")
    for index, row in df_email_contacts.iterrows():
        if row['LE'] == cocd:
            #--Send file by email:
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = row['Emails2']
            mail.Subject = f"Automatic email for {row['LE']}"
            mail.Body = f"Please find attached a file with TP extract for your Cocd {row['LE']}"
            mail.Attachments.Add(save_path)
            #mail.Display(True)
            #mail.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Emails\\Email to {row['LE']}.msg") # this one will save the mails in the unsaved state. And if put below mail.Send() will crush, for the item is already sent
            mail.Send()
            # time.sleep(3) # give enough time for the mail to appear in the mailbox
            # mapi = outlook.GetNameSpace("MAPI")
            # # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
            # folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
            # items= folder.Items
            # items.Sort("[ReceivedTime]", Descending=False)
            # msgs = items.GetLast()
            # msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\Email to {row['LE']}.msg")
            # print(msgs)
        else: pass

def send_emails_me_non_flash(save_path, cocd):
    # same for the non flash
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="non_flash_copy")
    for index, row in df_email_contacts.iterrows():
        if row['LE'] == cocd:
            #--Send file by email:
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = row['Emails2']
            mail.Subject = f"Automatic email for {row['LE']}"
            mail.Body = f"Please find attached a file with TP extract for your Cocd {row['LE']}"
            mail.Attachments.Add(save_path)
            #mail.Display(True)
            #mail.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Emails\\Email to {row['LE']}.msg") # this one will save the mails in the unsaved state. And if put below mail.Send() will crush, for the item is already sent
            mail.Send()
            # time.sleep(3) # give enough time for the mail to appear in the mailbox
            # mapi = outlook.GetNameSpace("MAPI")
            # # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
            # folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
            # items= folder.Items
            # items.Sort("[ReceivedTime]", Descending=False)
            # msgs = items.GetLast()
            # msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\Email to {row['LE']}.msg")
            # print(msgs) 
        else: pass

def send_emails_me_original(save_path):
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="flash_copy")
    for index, row in df_email_contacts.iterrows():
        print(row['LE'], row['Emails2'])
        #--Send file by email:
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        mail.To = row['Emails2']
        mail.Subject = f"Automatic email for {row['LE']}"
        mail.Body = f"Please find attached a file with TP extract for your Cocd {row['LE']}"
        mail.Attachments.Add(save_path)
        #mail.Display(True)
        #mail.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Emails\\Email to {row['LE']}.msg") # this one will save the mails in the unsaved state. And if put below mail.Send() will crush, for the item is already sent
        mail.Send()
        time.sleep(2) # give enough time for the mail to appear in the mailbox
        mapi = outlook.GetNameSpace("MAPI")
        # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
        folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
        items= folder.Items
        items.Sort("[ReceivedTime]", Descending=False)
        msgs = items.GetLast()
        msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\Email to {row['LE']}.msg")
        print(msgs)
    # same for the non flash
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="non_flash_copy")
    for index, row in df_email_contacts.iterrows():
        print(row['LE'], row['Emails2'])
        #--Send file by email:
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        mail.To = row['Emails2']
        mail.Subject = f"Automatic email for {row['LE']}"
        mail.Body = f"Please find attached a file with TP extract for your Cocd {row['LE']}"
        #mail.Attachments.Add(entry_path.get())
        #mail.Display(True)
        #mail.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Emails\\Email to {row['LE']}.msg") # this one will save the mails in the unsaved state. And if put below mail.Send() will crush, for the item is already sent
        mail.Send()
        time.sleep(2) # give enough time for the mail to appear in the mailbox
        mapi = outlook.GetNameSpace("MAPI")
        # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
        folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
        items= folder.Items
        items.Sort("[ReceivedTime]", Descending=False)
        msgs = items.GetLast()
        msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\Email to {row['LE']}.msg")
        print(msgs)
        
sap_extract_me()
excel_rework_me()

Automatic email for 7927
